In [1]:
%pip install tiktoken pymupdf pymupdf4llm -q

Note: you may need to restart the kernel to use updated packages.


# 필수 패키지 설치

In [3]:
# 환경 설정 및 모듈 임포트
import sys
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.processors.document_processor import DocumentProcessor
from src.db.documents_db import DocumentsDB
from src.utils.logging_config import get_logger

logger = get_logger(__name__)
logger.info("테스트 환경 설정 완료")

2025-11-21 14:00:30 [I] __main__ - 테스트 환경 설정 완료


In [4]:
%pip install --upgrade helper-hwp

Note: you may need to restart the kernel to use updated packages.


## HWP, PDF 혼입해서 DB 저장 테스트

In [5]:
def test_documents_processing() -> None:
    """PDF 및 HWP 파일 처리 테스트"""
    
    # data/raw 디렉토리 설정 (현재는 hwp, pdf 혼합 테스트)
    raw_dir = project_root / "data" / "raw"
    
    if not raw_dir.exists():
        logger.warning(f"디렉토리를 찾을 수 없습니다: {raw_dir}")
        return

    # PDF와 HWP 파일 모두 검색
    target_extensions = ['*.pdf', '*.hwp']
    target_files = []
    for ext in target_extensions:
        target_files.extend(list(raw_dir.glob(ext)))
    
    if not target_files:
        logger.warning(f"⚠ 지정된 경로({raw_dir})에 PDF 또는 HWP 파일이 없습니다.")
        return
    
    logger.info(f"발견된 처리 대상 파일: **{len(target_files)}**개")
    
    # DocumentProcessor 인스턴스 생성
    try:
        db_path = str(project_root / "data" / "test_documents_mixed.db")
        # 매 테스트마다 삭제 후 재생성을 원하면 아래 주석 해제
        if Path(db_path).exists():
            Path(db_path).unlink()
            logger.info(f"기존 DB 파일 삭제됨: {db_path}")

        processor = DocumentProcessor(db_path=db_path)
    except Exception as e:
        logger.error(f"✗ DocumentProcessor 초기화 중 오류: {e}")
        return

    count = 1
    # --- 파일 처리 반복문 ---
    for file_path in target_files:
        logger.info(f"{len(target_files)}개 중 {count}번째 파일 처리: **{file_path.name}**")
        
        try:
            # process_pdf -> process_file 로 변경됨
            file_hash = processor.process_doc(str(file_path))
            
            if file_hash:                
                # 검증: DB 조회
                saved_doc = processor.docs_db.get_document_by_hash(file_hash)
                if saved_doc:
                    doc_type = "PDF" if file_path.suffix.lower() == '.pdf' else "HWP"
                    logger.info(f"✓ [{doc_type}] 저장 검증 완료: {file_hash[:16]}... (len: {len(saved_doc.get('text_content', ''))})")
                else:
                    logger.error("✗ 저장 후 조회 실패")
            else:
                # process_file 내부에서 이미 에러 로깅을 했을 수 있음 (예: 미지원 포맷 등)
                logger.warning(f"⚠ 파일 처리 결과 없음 (Skip 또는 오류): {file_path.name}")
                
        except Exception as e:
            logger.error(f"✗ 파일 처리 중 예외 발생 [{file_path.name}]: {e}")
        
        count += 1
            
    logger.info("모든 문서 파일 처리가 완료되었습니다.")
    
    # 최종 통계 출력
    stats = processor.docs_db.get_document_stats()
    logger.info(f"최종 DB 통계: {stats}")

# 실행
if __name__ == "__main__":
    test_documents_processing()

2025-11-21 14:01:02 [I] __main__ - 발견된 처리 대상 파일: **100**개
2025-11-21 14:01:02 [I] __main__ - 기존 DB 파일 삭제됨: c:\_codeit_mid\codeit_ai_g2b_search\data\test_documents_mixed.db
2025-11-21 14:01:03 [I] [DOCDB] - 데이터베이스 버전 초기화: 1
2025-11-21 14:01:03 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: c:\_codeit_mid\codeit_ai_g2b_search\data\test_documents_mixed.db)
2025-11-21 14:01:03 [I] __main__ - 100개 중 1번째 파일 처리: **고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf**
2025-11-21 14:01:03 [I] [DOCP] - PDF 처리 시작: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
2025-11-21 14:01:03 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 65c9785a157c8f554795a39912bac9b7e32d3431810d560ca0847d4ccdecc2ea)


PDF to Markdown: 100%|██████████| 297/297 [11:20<00:00,  2.29s/page, 페이지 297 len=738]  

2025-11-21 14:12:24 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-21 14:12:24 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-21 14:12:24 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf
2025-11-21 14:12:24 [I] __main__ - ✓ [PDF] 저장 검증 완료: 65c9785a157c8f55... (len: 273661)
2025-11-21 14:12:24 [I] __main__ - 100개 중 2번째 파일 처리: **기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf**
2025-11-21 14:12:24 [I] [DOCP] - PDF 처리 시작: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
2025-11-21 14:12:24 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce)



PDF to Markdown: 100%|██████████| 49/49 [01:25<00:00,  1.75s/page, 페이지 49 len=358] 

2025-11-21 14:13:49 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-21 14:13:49 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-21 14:13:50 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
2025-11-21 14:13:50 [I] __main__ - ✓ [PDF] 저장 검증 완료: 56e5a8ae813dbddf... (len: 67438)
2025-11-21 14:13:50 [I] __main__ - 100개 중 3번째 파일 처리: **서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf**
2025-11-21 14:13:50 [I] [DOCP] - PDF 처리 시작: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf
2025-11-21 14:13:50 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: cd81e943ce28db811cbad55454b5fd406b27d0660dbb2bcfe521cc05fc7b7576)


...

2025-11-21 14:21:26 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-21 14:21:26 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-21 14:21:26 [I] [DOCP] - HWP 처리 및 저장 완료: 한국철도공사 (용역)_예약발매시스템 개량 ISMP 용역.hwp
2025-11-21 14:21:26 [I] __main__ - ✓ [HWP] 저장 검증 완료: 79c3ff04b9a8d404... (len: 85968)
2025-11-21 14:21:26 [I] __main__ - 100개 중 98번째 파일 처리: **한국한의학연구원_통합정보시스템 고도화 용역.hwp**
2025-11-21 14:21:26 [I] [DOCP] - HWP 처리 시작: 한국한의학연구원_통합정보시스템 고도화 용역.hwp
2025-11-21 14:21:26 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: fe07779f264abfd5f420f332adb65779a33142a5506581345f5cbe4803be53e8)
2025-11-21 14:21:26 [I] [DOCP] - 이미 처리된 파일 (skip): 한국한의학연구원_통합정보시스템 고도화 용역.hwp
2025-11-21 14:21:26 [I] __main__ - ✓ [HWP] 저장 검증 완료: fe07779f264abfd5... (len: 71630)
2025-11-21 14:21:26 [I] __main__ - 100개 중 99번째 파일 처리: **한국해양조사협회_2024년 항해용 간행물 품질관리 업무보조 시스템 구축.hwp**
2025-11-21 14:21:26 [I] [DOCP] - HWP 처리 시작: 한국해양조사협회_2024년 항해용 간행물 품질관리 업무보조 시스템 구축.hwp
2025-11-21 14:21:26 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 2999dd9e2f06db1b31d0ece80f69a318


HWP to Markdown 80줄마다 페이지 분할: 100%|██████████| 26/26 [00:00<00:00, 1368.06page/s, 페이지 26 len=1291]

2025-11-21 14:21:27 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-21 14:21:27 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-21 14:21:27 [I] [DOCP] - HWP 처리 및 저장 완료: 한국해양조사협회_2024년 항해용 간행물 품질관리 업무보조 시스템 구축.hwp
2025-11-21 14:21:27 [I] __main__ - ✓ [HWP] 저장 검증 완료: 2999dd9e2f06db1b... (len: 69322)
2025-11-21 14:21:27 [I] __main__ - 100개 중 100번째 파일 처리: **한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp**
2025-11-21 14:21:27 [I] [DOCP] - HWP 처리 시작: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
2025-11-21 14:21:27 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 1d7e80b4f717de629d0917b5e1412ffb7d913fb692bf3f1ec8eb4964b276e635)



HWP to Markdown 80줄마다 페이지 분할: 100%|██████████| 21/21 [00:00<00:00, 1399.70page/s, 페이지 21 len=1570]

2025-11-21 14:21:28 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-21 14:21:28 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-21 14:21:28 [I] [DOCP] - HWP 처리 및 저장 완료: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
2025-11-21 14:21:28 [I] __main__ - ✓ [HWP] 저장 검증 완료: 1d7e80b4f717de62... (len: 50283)
2025-11-21 14:21:28 [I] __main__ - 모든 문서 파일 처리가 완료되었습니다.
2025-11-21 14:21:28 [I] __main__ - 최종 DB 통계: {'total_files': 98, 'total_pages': 3056, 'total_size_bytes': 169091453, 'total_size_mb': 161.26}


#### 아래 코드 실행하여 hwp 파일 1개 처리하여 생성된 마크다운 내용 평가
- 페이지 번호를 가져오는 것은 무리인가? 처리하는 과정에서 hwp가 가지고 있는 페이지 분할 지점에 대한 정보를 안정적으로 카운트 해서 페이지 번호에 반영할 수 있는지 보장이 필요하다.
- 목차를 가져오지는 못했고, 해당 페이지 내용 생략했음, 목차에 굳이 집착해야 하는가?라는 생각이 들어서 우선 보류
- 법률 조항을 가져오는데 문제 없음 "제*조의 *에 따라" 부분의 *를 잘 가져옴
- 
#### PDF쪽 문제점과 HWP 마크다운 결과 비교
- 문장의 글머리가 특이하면 첫 줄을 받아오지 않는 문제가 있었는데 hwp쪽은 그것이 해결되었음
- 법률 조항을 가져오는데 "제*조의 *"의 숫자들을 정상적으로 잘 받아옴
- PDF쪽은 한자를 못 가져오는데 HWP는 한자를 잘 가져옴

## 샘플 파일 1개로 hwp 마크다운 변환하고 결과 확인

In [ ]:
from pathlib import Path

from helper_hwp import hwp_to_markdown

def test_real_hwp_if_available() -> None:
    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data"
    hwp_files = list(raw_dir.glob("*.hwp")) if raw_dir.exists() else []

    if not hwp_files:
        logger.warning("⚠ data/raw 디렉토리에 hwp 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. hwp 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 hwp 파일: {len(hwp_files)}개")
    
    # 첫 번째 PDF 파일 처리
    hwp_path = hwp_files[0]
    logger.info(f"처리 중: {hwp_path.name}")

    if not hwp_path.exists():
        logger.warning(f'샘플 HWP 파일 없음: {hwp_path}')
    else:
        markdown = hwp_to_markdown(str(hwp_path))
        logger.info(f'변환 완료: {len(markdown)} chars')
        logger.debug(markdown[:1000])
        md_path = Path('test1.md')
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write(markdown)
        logger.info(f'Markdown 파일 저장 완료: {md_path}')

# 실행
test_real_hwp_if_available()

## PDF 다른 라이브러리 활용해서 마크다운 수행해보기

In [ ]:
# command window에서 진행
# 기본 unstructured 패키지 설치
# pip install unstructured

# PDF 처리를 위한 필수 종속성 설치 (pypdfium2, pillow 등 포함)
# pip install unstructured-pytesseract unstructured-pdf

## PDF 파일 하나만 넣기

In [6]:
# 하나만 넣기
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path
    
    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "pdf_conv_2"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: {len(pdf_files)}개")
    
    # 첫 번째 PDF 파일 처리
    pdf_path = pdf_files[0]
    logger.info(f"처리 중: {pdf_path.name}")
    
    try:
        processor = DocumentProcessor(db_path=str(project_root / "data" / "test_documents.db"))
        file_hash = processor.process_pdf(str(pdf_path))
        
        if file_hash:
            logger.info(f"✓ PDF 처리 성공: {file_hash[:16]}...")
        else:
            logger.error("✗ PDF 처리 실패 (PyMuPDF 미설치 가능성)")
            
    except Exception as e:
        logger.error(f"✗ PDF 처리 중 오류: {e}")

# 실행
test_real_pdf_if_available()

2025-11-14 14:00:26 [I] __main__ - 발견된 PDF 파일: 100개
2025-11-14 14:00:26 [I] __main__ - 처리 중: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents.db)
2025-11-14 14:00:26 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-14 14:00:26 [D] [DOCP] - 파일 해시: 5a28516e24903a44...


PDF to Markdown:   0%|          | 0/96 [00:00<?, ?page/s]

2025-11-14 14:01:45 [I] [DOCP] - PDF 처리 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf (96 페이지, 1675.0KB), 변환문자수=116,963
2025-11-14 14:01:45 [I] __main__ - ✓ PDF 처리 성공: 5a28516e24903a44...


## 전체 파일 로드해서 test_documents.db에 넣어보기

In [ ]:
def test_real_pdf_if_available() -> None:
    """실제 PDF 파일이 있는 경우 테스트"""
    from pathlib import Path

    # data/raw 디렉토리에서 PDF 파일 검색
    raw_dir = project_root / "data" / "pdf_conv_3"
    pdf_files = list(raw_dir.glob("*.pdf")) if raw_dir.exists() else []
    
    if not pdf_files:
        logger.warning("⚠ data/raw 디렉토리에 PDF 파일이 없습니다.")
        logger.info("테스트를 건너뜁니다. PDF 파일을 추가하려면:")
        logger.info(f"  경로: {raw_dir}")
        return
    
    logger.info(f"발견된 PDF 파일: **{len(pdf_files)}**개")
    
    # DocumentProcessor 인스턴스를 루프 밖에서 한 번만 생성
    try:
        db_path = str(project_root / "data" / "test_documents_3.db")
        # DB 파일이 이미 존재하면 삭제 (스키마 충돌 방지)
        if Path(db_path).exists():
            Path(db_path).unlink()
            logger.info(f"기존 DB 파일 삭제됨: {db_path}")
            
        processor = DocumentProcessor(db_path=db_path)
    except Exception as e:
        logger.error(f"✗ DocumentProcessor 초기화 중 오류: {e}")
        return

    count = 1 # 문서 처리 개수 세는 카운트
    # --- 모든 PDF 파일을 처리하는 반복문 구조 ---
    for pdf_path in pdf_files:
        logger.info(f"{len(pdf_files)}개 파일 중 {count}번째 파일 처리 중:  **{pdf_path.name}**")
        
        try:
            file_hash = processor.process_pdf(str(pdf_path))
            
            if file_hash:                
                # 검증: DB에서 다시 불러와서 길이가 맞는지 확인
                saved_doc = processor.docs_db.get_document_by_hash(file_hash)
                if saved_doc:
                    logger.info(f"✓ PDF 처리 후 저장 검증 완료: {file_hash[:16]}...")
                else:
                    logger.error("✗ 저장 후 조회 실패")
            else:
                # process_pdf가 성공했지만 해시를 반환하지 않은 경우 (예: 파일이 비어 있음)
                logger.error(f"✗ PDF 처리 실패 (파일 해시 반환 안 됨): {pdf_path.name}")
                
        except Exception as e:
            # PyMuPDF 미설치 등 구체적인 오류를 로깅
            logger.error(f"✗ PDF 처리 중 오류 [{pdf_path.name}]: {e}")
        count += 1
            
    logger.info("모든 PDF 파일 처리가 완료되었습니다.")

# 실행
test_real_pdf_if_available()

2025-11-19 10:58:23 [I] __main__ - 발견된 PDF 파일: **100**개
2025-11-19 10:58:23 [I] [DOCDB] - 데이터베이스 버전 초기화: 1
2025-11-19 10:58:23 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: /home/spai0438/codeit_ai_g2b_search/data/test_documents_3.db)
2025-11-19 10:58:23 [I] __main__ - 100개 파일 중 1번째 파일 처리 중:  **한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf**
2025-11-19 10:58:23 [I] [DOCP] - PDF 처리 시작: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-19 10:58:23 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 4343a1664887358147d0fd51982ca273ac5d500e966b2cfcb990e1ac2437bb0f)


PDF to Markdown: 100%|██████████| 96/96 [01:04<00:00,  1.48page/s, 페이지 96 len=767] 

2025-11-19 10:59:28 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-19 10:59:28 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-19 10:59:28 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 한국가스공사_[재공고]차세대 통합정보시스템(ERP) 구축.pdf
2025-11-19 10:59:28 [I] __main__ - ✓ PDF 처리 후 저장 검증 완료: 4343a16648873581...
2025-11-19 10:59:28 [I] __main__ - 100개 파일 중 2번째 파일 처리 중:  **광주과학기술원_학사시스템 기능개선 사업.pdf**
2025-11-19 10:59:28 [I] [DOCP] - PDF 처리 시작: 광주과학기술원_학사시스템 기능개선 사업.pdf
2025-11-19 10:59:28 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: ab7378a7405213f53cf45a4f4ca4aabf22a38363c3331cadc9fddee266944e2f)



PDF to Markdown:   0%|          | 0/66 [00:00<?, ?page/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



...

2025-11-19 13:27:17 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-19 13:27:17 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-19 13:27:17 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 서울특별시 여성가족재단_(재공고, 협상) 서울 디지털성범죄 안심지원센.pdf
2025-11-19 13:27:17 [I] __main__ - ✓ PDF 처리 후 저장 검증 완료: 24eb9a94d17e678c...
2025-11-19 13:27:17 [I] __main__ - 100개 파일 중 99번째 파일 처리 중:  **재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf**
2025-11-19 13:27:17 [I] [DOCP] - PDF 처리 시작: 재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf
2025-11-19 13:27:17 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: 861bfd25152d39f2989105bfc26c304052e99f406b77a402a1e70101d37e29b9)



PDF to Markdown: 100%|██████████| 137/137 [01:33<00:00,  1.47page/s, 페이지 137 len=532] 

2025-11-19 13:28:50 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-19 13:28:50 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-19 13:28:50 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 재단법인경기도일자리재단_2025년 통합접수시스템 운영.pdf
2025-11-19 13:28:50 [I] __main__ - ✓ PDF 처리 후 저장 검증 완료: 861bfd25152d39f2...
2025-11-19 13:28:50 [I] __main__ - 100개 파일 중 100번째 파일 처리 중:  **한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf**
2025-11-19 13:28:50 [I] [DOCP] - PDF 처리 시작: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf
2025-11-19 13:28:50 [D] [DOCDB] - 검색 완료: 0건 (hash 모드, 검색어: a47caca5184d9ecaf7bd6099cac61a2961f75ece3f7f9540a6bdabca2dbd5363)



PDF to Markdown: 100%|██████████| 74/74 [00:56<00:00,  1.31page/s, 페이지 74 len=723] 

2025-11-19 13:29:47 [I] [DOCP] - 텍스트 분할 저장 시작 (총 5개 조각)
2025-11-19 13:29:47 [I] [DOCP] - 마커 덤프 생성 완료
2025-11-19 13:29:47 [I] [DOCP] - PDF 처리 및 분할 저장 완료: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.pdf
2025-11-19 13:29:47 [I] __main__ - ✓ PDF 처리 후 저장 검증 완료: a47caca5184d9eca...
2025-11-19 13:29:47 [I] __main__ - 모든 PDF 파일 처리가 완료되었습니다.


## 5. 실제 PDF 파일 테스트 (선택사항)

In [12]:
def verify_database_content() -> None:
    """DB 내용 확인 및 검증"""
    import sqlite3
    
    test_db_path = project_root / "data" / "test_documents.db"
    
    with sqlite3.connect(test_db_path) as conn:
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        
        # 1. 파일 정보 조회
        # 모든 테이블 이름 조회 쿼리
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        logger.info(f"모든 테이블 이름 조회 쿼리 {cursor.fetchall()}")
        files = cursor.fetchall()
        logger.info(f"\n=== 파일 정보 ({len(files)}건) ===")
        for row in files:
            logger.info(f"  - {row['file_name']}: {row['total_pages']}페이지, {row['total_tokens']}토큰")
        
        # 2. 페이지 데이터 조회
        # cursor.execute("""
        #     SELECT file_hash, page_number, is_empty 
        #     FROM page_data 
        #     ORDER BY file_hash, page_number
        # """)
        pages = cursor.fetchall()
        logger.info(f"\n=== 페이지 데이터 ({len(pages)}건) ===")
        
        # 파일별로 그룹화
        from collections import defaultdict
        pages_by_file = defaultdict(list)
        for row in pages:
            pages_by_file[row['file_hash']].append(row)
        
        for file_hash, file_pages in pages_by_file.items():
            logger.info(f"  파일: {file_hash[:16]}...")
            for page in file_pages:
                status = "빈페이지" if page['is_empty'] else "정상"
                logger.info(f"    - 페이지 {page['page_number']}: {page['token_count']}토큰 ({status})")
        
        # 3. 통계 조회
        db = DocumentsDB(str(test_db_path))
        stats = db.get_document_stats()
        logger.info(f"\n=== 전체 통계 ===")
        logger.info(f"  총 파일: {stats['total_files']}개")
        logger.info(f"  총 페이지: {stats['total_pages']}개")
        # logger.info(f"  총 토큰: {stats['total_tokens']:,}개")
        logger.info(f"  총 크기: {stats['total_size_mb']}MB")
        
        logger.info("\n✓ DB 검증 완료")

# 실행
verify_database_content()

2025-11-14 14:20:49 [I] __main__ - 모든 테이블 이름 조회 쿼리 [<sqlite3.Row object at 0x7a2692c3f6a0>]
2025-11-14 14:20:49 [I] __main__ - 
=== 파일 정보 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 페이지 데이터 (0건) ===
2025-11-14 14:20:49 [I] __main__ - 
=== 전체 통계 ===
2025-11-14 14:20:49 [I] __main__ -   총 파일: 1개
2025-11-14 14:20:49 [I] __main__ -   총 페이지: 96개
2025-11-14 14:20:49 [I] __main__ -   총 크기: 1.64MB
2025-11-14 14:20:49 [I] __main__ - 
✓ DB 검증 완료


## 4. DB 조회 및 검증

In [9]:
def test_pdf_processing_simulation() -> None:
    """PDF 처리 로직 시뮬레이션 (PyMuPDF 없이)"""
    
    # 실제 PDF 대신 더미 데이터로 로직 테스트
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # PDF 페이지 시뮬레이션
    dummy_pages = [
        "# 제목 페이지\n\n이것은 첫 번째 페이지입니다.",
        "# 본문 1\n\n두 번째 페이지 내용입니다.",
        "",  # 빈 페이지
        "# 결론\n\n마지막 페이지입니다."
    ]
    
    # 파일 정보 시뮬레이션
    dummy_hash = "dummy_pdf_hash_67890"
    dummy_filename = "test_dummy.pdf"
    
    logger.info(f"PDF 처리 시뮬레이션: {dummy_filename}")
    
    # 페이지별 처리
    all_content = []
    for page_num, page_text in enumerate(dummy_pages, start=1):
        is_empty = len(page_text.strip()) < 10
        
        if is_empty:
            page_content = "[빈 페이지]"
        else:
            page_content = f"--- 페이지 {page_num} ---\n\n{page_text}"
        
        all_content.append(page_content)
        
        # DB 저장
        token_count = processor.count_tokens(page_content)
        processor.docs_db.insert_page_data(
            file_hash=dummy_hash,
            page_number=page_num,
            markdown_content=page_content,
            token_count=token_count,
            is_empty=is_empty
        )
        logger.info(f"페이지 {page_num}: {token_count} 토큰, 빈페이지={is_empty}")
    
    # 전체 콘텐츠
    full_content = "\n\n".join(all_content)
    total_tokens = processor.count_tokens(full_content)
    
    # 파일 정보 저장
    processor.docs_db.insert_file_info(
        file_hash=dummy_hash,
        file_name=dummy_filename,
        total_pages=len(dummy_pages),
        file_size=len(full_content.encode('utf-8')),
        total_chars=len(full_content),
        total_tokens=total_tokens
    )
    
    logger.info(f"✓ PDF 시뮬레이션 완료: {len(dummy_pages)}페이지, {total_tokens}토큰")

# 테스트 실행
test_pdf_processing_simulation()

2025-11-08 22:36:33 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - PDF 처리 시뮬레이션: test_dummy.pdf
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 1: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 2: 22 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - 페이지 3: 6 토큰, 빈페이지=True
2025-11-08 22:36:33 [I] __main__ - 페이지 4: 19 토큰, 빈페이지=False
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰
2025-11-08 22:36:33 [I] __main__ - ✓ PDF 시뮬레이션 완료: 4페이지, 69토큰


## 3. PDF 처리 테스트 (PyMuPDF 없이 시뮬레이션)

In [8]:
def test_document_processor_utils() -> None:
    """DocumentProcessor 유틸리티 함수 테스트"""
    import tempfile
    
    # DocumentProcessor 초기화
    test_db_path = project_root / "data" / "test_documents.db"
    processor = DocumentProcessor(db_path=str(test_db_path))
    
    # 1. 임시 파일 생성
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False, encoding='utf-8') as tf:
        test_content = "This is a test document for hash calculation."
        tf.write(test_content)
        temp_file_path = Path(tf.name)
    
    try:
        # 2. 파일 해시 계산
        file_hash = processor.calculate_file_hash(temp_file_path)
        logger.info(f"파일 해시: {file_hash[:16]}...")
        assert len(file_hash) == 64, "SHA-256 해시 길이 오류"
        
        # 3. 토큰 카운트
        token_count = processor.count_tokens(test_content)
        logger.info(f"토큰 수: {token_count}")
        assert token_count > 0, "토큰 카운트 오류"
        
        logger.info("✓ 유틸리티 함수 테스트 성공")
        
    finally:
        # 임시 파일 삭제
        if temp_file_path.exists():
            temp_file_path.unlink()

# 테스트 실행
test_document_processor_utils()

2025-11-08 22:36:26 [I] src.processors.document_processor - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db)
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공
2025-11-08 22:36:26 [I] __main__ - 파일 해시: 258983484b09a700...
2025-11-08 22:36:26 [I] __main__ - 토큰 수: 9
2025-11-08 22:36:26 [I] __main__ - ✓ 유틸리티 함수 테스트 성공


## 2. DocumentProcessor 해시 계산 및 토큰 카운트 테스트

In [7]:
def test_documents_db_crud() -> None:
    """DocumentsDB CRUD 테스트"""
    from datetime import datetime
    import sqlite3
    
    # 테스트용 DB 경로
    test_db_path = project_root / "data" / "test_documents.db"
    test_db_path.parent.mkdir(parents=True, exist_ok=True)
    
    # DB 초기화
    db = DocumentsDB(str(test_db_path))
    
    # 기존 테스트 데이터 삭제 (파일 삭제 대신)
    with sqlite3.connect(test_db_path) as conn:
        conn.execute("DELETE FROM page_data")
        conn.execute("DELETE FROM file_info")
        conn.commit()
    
    logger.info(f"테스트 DB 초기화: {test_db_path}")
    
    # 1. 파일 정보 삽입 (Create)
    test_hash = "test_hash_12345"
    success = db.insert_file_info(
        file_hash=test_hash,
        file_name="test_document.pdf",
        total_pages=3,
        file_size=1024 * 50,  # 50KB
        total_chars=5000,
        total_tokens=1200
    )
    logger.info(f"파일 정보 삽입: {success}")
    
    # 2. 페이지 데이터 삽입
    for page_num in range(1, 4):
        content = f"# 페이지 {page_num}\n\n테스트 콘텐츠입니다."
        db.insert_page_data(
            file_hash=test_hash,
            page_number=page_num,
            markdown_content=content,
            token_count=100 + page_num * 10,
            is_empty=False
        )
    logger.info("페이지 데이터 삽입 완료")
    
    # 3. 통계 조회 (Read)
    stats = db.get_document_stats()
    logger.info(f"문서 통계: {stats}")
    
    # 4. 데이터 확인
    assert stats['total_files'] >= 1, "파일 개수 불일치"
    assert stats['total_pages'] >= 3, "페이지 개수 불일치"
    
    logger.info("✓ CRUD 테스트 성공")
    return db, test_hash

# 테스트 실행
db, test_hash = test_documents_db_crud()

2025-11-08 22:36:20 [I] __main__ - 테스트 DB 초기화: d:\GoogleDrive\codeit_ai_g2b_search\data\test_documents.db
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 파일 정보 삽입: True
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공
2025-11-08 22:36:20 [I] __main__ - 페이지 데이터 삽입 완료
2025-11-08 22:36:20 [I] __main__ - 문서 통계: {'total_files': 1, 'total_pages': 3, 'total_tokens': 1200, 'total_size_bytes': 51200, 'total_size_mb': 0.05}
2025-11-08 22:36:20 [I] __main__ - ✓ CRUD 테스트 성공


## 1. DocumentsDB 기본 CRUD 테스트

## 테스트 요약

위의 테스트들을 통해 다음을 확인했습니다:

1. **DocumentsDB CRUD**: 파일 정보/페이지 데이터 삽입 및 조회 정상 동작
2. **DocumentProcessor 유틸리티**: 파일 해시 계산, 토큰 카운트 정상
3. **PDF 처리 시뮬레이션**: 페이지별 콘텐츠 처리 로직 검증
4. **DB 검증**: 전체 통계 및 데이터 무결성 확인
5. **실제 PDF 처리**: PyMuPDF 사용 가능 시 자동 처리

**추가 테스트 방법**:
- `data/raw/` 디렉토리에 PDF 파일을 추가하면 실제 PDF 처리 테스트 자동 실행됩니다.
- HWP 파일 처리는 현재 `document_processor.py`에 구현되지 않았으므로 향후 추가 필요합니다.